In [1]:
%classpath add jar ../../konduit.jar

In [2]:
package ai.konduit.serving;

import ai.konduit.serving.endpoint.Endpoint;

import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Timer;
import java.util.TimerTask;
import io.vertx.core.http.HttpHeaders;

import java.io.File;

import java.nio.charset.StandardCharsets;
import org.apache.commons.io.FileUtils;
import io.vertx.core.http.HttpHeaders;
import io.vertx.ext.web.handler.StaticHandler;

import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class WebAppEndpoint implements Endpoint {
    
    final static Logger logger = LoggerFactory.getLogger(WebAppEndpoint.class);

    private PipelineExecutor pipelineExecutor;

    public WebAppEndpoint(PipelineExecutor pipelineExecutor) {
        this.pipelineExecutor = pipelineExecutor;
    }

    public HttpMethod type() { return HttpMethod.GET; }

    public String path() { return "/web-app/*"; }

    public List<String> consumes() { return Arrays.asList(); }

    public List<String> produces() { return Arrays.asList("application/html"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> { 
            try {
                logger.info(new File(handler.request().path().substring(1)).getAbsolutePath());
                handler.response().sendFile(new File(handler.request().path().substring(1)).getAbsolutePath()).end(); 
            } catch(Exception e) {
                e.printStackTrace();
                logger.error("Error: ", e);
            }
        };
    }
}

ai.konduit.serving.WebAppEndpoint

In [3]:
package ai.konduit.serving;

import ai.konduit.serving.endpoint.Endpoint;

import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import io.micrometer.prometheus.PrometheusConfig;
import io.micrometer.core.instrument.MeterRegistry;
import io.micrometer.prometheus.PrometheusMeterRegistry;
import io.micrometer.core.instrument.Counter;
import io.micrometer.core.instrument.binder.jvm.JvmMemoryMetrics;
import io.micrometer.core.instrument.binder.system.ProcessorMetrics;

import java.util.Timer;
import java.util.TimerTask;
import io.vertx.core.http.HttpHeaders;

public class PrometheusEndpoint implements Endpoint {

    public static PrometheusMeterRegistry registry;
    public static List<Counter> classCounterIncrement = new ArrayList();
    public static List<String> labels = Arrays.asList("UnderWeight", "Normal_Range", "OverWeight", "Obese_ClassI", "Obese_ClassII", "Obese_ClassIII", "Obese_ClassIV");
    
    static {
        registry = new PrometheusMeterRegistry(PrometheusConfig.DEFAULT);
        
        if (registry != null) {
            System.out.println("Using metrics registry " + registry.getClass().getName() + " for inference");
            new JvmMemoryMetrics().bindTo(registry);
            new ProcessorMetrics().bindTo(registry);
            
            // For scraping GPU metrics
            try {
                Class<?> gpuMetricsClass = Class.forName("ai.konduit.serving.gpu.GpuMetrics");
                Object instance = gpuMetricsClass.newInstance();
                gpuMetricsClass.getMethod("bindTo", MeterRegistry.class).invoke(instance, registry);
            } catch(Exception exception) {
                System.out.println("No GPU binaries found. Selecting and scraping only CPU metrics.");
            }
            
            Counter serverUpTimeCounter = registry.counter("bmi-onnx-pytorch.server.up.time");
            double increment = 5.0;
            new Timer().schedule(new TimerTask() {
                @Override
                public void run() {
                    serverUpTimeCounter.increment(increment);
                }
            }, 5000, ((int) increment) * 1000);
            
            
            for (String label : labels) {
                classCounterIncrement.add(Counter.builder(label)
                        .description("Classification counts seen so far for class label: " + label)
                        .baseUnit("bmi-onnx-pytorch.classification.outcome")
                        .register(registry));
            }
        } else {
            System.out.println("Not using metrics registry.");
        }
    }
    
    private PipelineExecutor pipelineExecutor;

    public PrometheusEndpoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor;
    }

    public HttpMethod type() { return HttpMethod.GET; }

    public String path() { return "/server-metrics"; }

    public List<String> consumes() { return Arrays.asList(); }

    public List<String> produces() { return Arrays.asList("text/plain; version=0.0.4; charset=utf-8"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> handler.response().putHeader(HttpHeaders.CONTENT_TYPE, "text/plain; version=0.0.4; charset=utf-8").end(registry.scrape());
    }
}

ai.konduit.serving.PrometheusEndpoint

In [4]:
package ai.konduit.serving;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import io.micrometer.core.instrument.Counter;
import io.micrometer.core.instrument.Gauge;

import ai.konduit.serving.pipeline.util.ObjectMappers;
import ai.konduit.serving.pipeline.registry.NDArrayConverterRegistry;
import ai.konduit.serving.data.nd4j.format.ND4JConverters;
import io.vertx.core.json.JsonObject;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    private double requestTime = -1.0;
    private double pipelineTime = -1.0;

    private Counter requestsHandled = PrometheusEndpoint.registry.counter("bmi-onnx-pytorch.requests.handled");
    private Gauge requestTimeGuage = Gauge.builder("bmi-onnx-pytorch.request.time.ms", () -> requestTime).register(PrometheusEndpoint.registry);
    private Gauge pipelineTimeGuage = Gauge.builder("bmi-onnx-pytorch.pipeline.time.ms", () -> pipelineTime).register(PrometheusEndpoint.registry);
    private Gauge requestThroughputGuage = Gauge.builder("bmi-onnx-pytorch.request.time.ms", () -> 1 / requestTime * 1000).register(PrometheusEndpoint.registry);

    public OCREndPoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor; 
        ImageFactoryRegistry.addFactory(new JavaImageFactory()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.Nd4jToSerializedConverter()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.SerializedToNd4jArrConverter());
    }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                double requestTimeStart = (double) System.currentTimeMillis();
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                
                    double pipelineTimeStart = (double) System.currentTimeMillis();
                    Data exec = pipelineExecutor.exec(image);
                    double pipelineTimeEnd = (double) System.currentTimeMillis();
                    pipelineTime = pipelineTimeEnd - pipelineTimeStart;
                    
                    handler.response().end(ObjectMappers.toJson(exec));
                    taskHandler.complete();

                    requestsHandled.increment();

                    String bmiClass = exec.getString("bmi_class");
                    int index = PrometheusEndpoint.labels.indexOf(bmiClass.replace(" ", "_").trim());
                    System.out.format("BMI CLASS: %s, for index %s", bmiClass, index);
                    if(index != -1) {
                        PrometheusEndpoint.classCounterIncrement.get(index).increment();
                    }
                    
                    double requestTimeEnd = (double) System.currentTimeMillis();
                    requestTime = requestTimeEnd - requestTimeStart;
                } catch (IOException e) {
                    e.printStackTrace();
                    handler.response().setStatusCode(500).end(new JsonObject().put("error", e.getMessage()).encode());
                    taskHandler.complete();
                    
                }
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.serving.OCREndPoint

In [5]:
package ai.konduit.serving;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {
    
    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor), new PrometheusEndpoint(pipelineExecutor), new WebAppEndpoint(pipelineExecutor));
    }
}

ai.konduit.serving.OCREndPoints

In [6]:
import java.net.URLClassLoader;
import java.net.URL;
import java.io.File;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.io.FileUtils;
import java.io.IOException;

import java.nio.charset.StandardCharsets;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();
List<String> classpaths = new ArrayList<>();

for(URL url : urls) {
    String singleClassPath = new File(url.toURI()).getAbsolutePath();
    System.out.println(singleClassPath);
    classpaths.add(singleClassPath);
}

try {
    String output = String.join(File.pathSeparator, classpaths);
    File classpathOutputPath = new File("classpath");
    FileUtils.writeStringToFile(new File("classpath"), output, StandardCharsets.UTF_8);
    System.out.format("%n-------------%nSaved content:%n-------------%n%s%n\tin file:%n%s%n-------------", output, classpathOutputPath.getAbsolutePath());
} catch (IOException e) {
    e.printStackTrace();
}

/tmp/beaker769589333114767497/outDir
/root/konduit/konduit.jar

-------------
Saved content:
-------------
/tmp/beaker769589333114767497/outDir:/root/konduit/konduit.jar
	in file:
/root/konduit/demos/6-bmi-onnx-pytorch/classpath
-------------

null

In [7]:
%%bash
nohup java -cp $(cat classpath) ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id bmi-onnx-pytorch -c bmi-onnx-pytorch.yaml -rwm &

In [9]:
%%bash
konduit logs bmi-onnx-pytorch -l 100

16:25:20.694 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /root/konduit/demos/6-bmi-onnx-pytorch/bmi-onnx-pytorch.yaml
16:25:20.708 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
16:25:20.710 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
16:25:20.711 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'bmi-onnx-pytorch'
16:25:21.466 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 28 PipelineStepRunnerFactory instances
16:25:22.058 [vert.x-worker-thread-0] INFO  a.k.serving.python.PythonRunner - Over riding python path :/root/miniconda/lib/python37.zip:/root/miniconda/lib/python3.7:/root/miniconda/lib/python3.7/lib-dynload:/root/miniconda/lib/python3.7/site-packages
16:25:25.185 [vert.x-worker-thread-0] INFO  a.k.serving.python.PythonRunner - Resolving

In [10]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | bmi-onnx-pytorch               | inference  | 0.0.0.0:9009         | 162     | started    




In [11]:
%%html
<img src="image_me.jpg"/>

In [12]:
%%bash
konduit inspect bmi-onnx-pytorch -q {port}

9009



In [13]:
%%bash
curl -s -H "Content-Type: multipart/form-data" -X POST -F "image=@image_me.jpg" http://localhost:$(konduit inspect bmi-onnx-pytorch -q {port})/infer

{
  "bmi_value" : 22.18,
  "bmi_class" : "Normal_Range",
  "boxes" : [ 447.0, 174.0, 636.0, 470.0 ]
}



## Metrics
The cell below also embeds the associated metrics...

In [14]:
%%html

<div style="display: flex; justify-content: center; align-items: center; border: 1px solid black;">
    <iframe src="http://localhost:3000/d/lP_JcnHWz/pipeline-metrics?orgId=1&refresh=5s&kiosk&var-serverName=bmi_onnx_pytorch" width=1500 height=1300>
</div>

## Web application
The cell below demonstrate the web application served by konduit-serving

In [15]:
%%html
<div style="display: flex; justify-content: center; align-items: center; border: 1px solid black;">
    <iframe src="http://localhost:9009/web-app/index.html" allow="camera;microphone", width=1000 height=1000></iframe>
</div>

In [16]:
%%bash
konduit stop bmi-onnx-pytorch

Stopping konduit server 'bmi-onnx-pytorch'
Application 'bmi-onnx-pytorch' terminated with status 0



## View metrics in Browser
Visit: http://localhost:3000/d/lP_JcnHWz/pipeline-metrics?orgId=1&refresh=5s&kiosk&var-serverName=tensorflow_mnist to view metrics.